In [48]:
import numpy as np 
import pandas as pd 
from datetime import date

In [49]:
ratios = [0,0.236, 0.382, 0.5 , 0.618, 0.786,1,1.414,1.618]
fiblevels = np.array(ratios)

In [196]:
def RSI(series, period=14):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    return 100 - 100 / (1 + rs)


# calculating Stoch RSI (gives the same values as TradingView)
# https://www.tradingview.com/wiki/Stochastic_RSI_(STOCH_RSI) 
def STRSIA(series, period=14, smoothK=3, smoothD=3):
    # Calculate RSI 
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    rsi = 100 - 100 / (1 + rs)

    # Calculate StochRSI 
    stochrsi  = (rsi - rsi.rolling(period).min()) / (rsi.rolling(period).max() - rsi.rolling(period).min())
    stochrsi_K = stochrsi.rolling(smoothK).mean()
    stochrsi_D = stochrsi_K.rolling(smoothD).mean()
    return stochrsi, stochrsi_K, stochrsi_D

In [197]:
ratios

[0, 0.236, 0.382, 0.5, 0.618, 0.786, 1, 1.414, 1.618]

In [198]:
ada15 = pd.read_json('BN-main/ADA_USDT-1h.json')
ada15

,0,1,2,3,4,5
0,1546300800000,0.04057,0.04059,0.03995,0.04035,5902479.60
1,1546304400000,0.04040,0.04048,0.03987,0.04000,3739370.50
2,1546308000000,0.04004,0.04004,0.03964,0.03991,3615775.00
3,1546311600000,0.03988,0.04021,0.03985,0.04020,2419828.60
4,1546315200000,0.04020,0.04068,0.04017,0.04043,5649221.70
...,...,...,...,...,...,...
22152,1626235200000,1.21680,1.23060,1.21220,1.22100,7593028.91
22153,1626238800000,1.22080,1.22170,1.19800,1.20280,15978748.32
22154,1626242400000,1.20280,1.22000,1.19980,1.20740,7789568.81
22155,1626246000000,1.20750,1.22410,1.20660,1.22200,6168385.93


In [199]:
ada15.columns=['date','open','high','low','close','volume']
ada15

,date,open,high,low,close,volume
0,1546300800000,0.04057,0.04059,0.03995,0.04035,5902479.60
1,1546304400000,0.04040,0.04048,0.03987,0.04000,3739370.50
2,1546308000000,0.04004,0.04004,0.03964,0.03991,3615775.00
3,1546311600000,0.03988,0.04021,0.03985,0.04020,2419828.60
4,1546315200000,0.04020,0.04068,0.04017,0.04043,5649221.70
...,...,...,...,...,...,...
22152,1626235200000,1.21680,1.23060,1.21220,1.22100,7593028.91
22153,1626238800000,1.22080,1.22170,1.19800,1.20280,15978748.32
22154,1626242400000,1.20280,1.22000,1.19980,1.20740,7789568.81
22155,1626246000000,1.20750,1.22410,1.20660,1.22200,6168385.93


In [200]:
start = pd.to_datetime('2020-01-01')
ada15.date=pd.to_datetime(ada15['date'],unit='ms')
ada15


,date,open,high,low,close,volume
0,2019-01-01 00:00:00,0.04057,0.04059,0.03995,0.04035,5902479.60
1,2019-01-01 01:00:00,0.04040,0.04048,0.03987,0.04000,3739370.50
2,2019-01-01 02:00:00,0.04004,0.04004,0.03964,0.03991,3615775.00
3,2019-01-01 03:00:00,0.03988,0.04021,0.03985,0.04020,2419828.60
4,2019-01-01 04:00:00,0.04020,0.04068,0.04017,0.04043,5649221.70
...,...,...,...,...,...,...
22152,2021-07-14 04:00:00,1.21680,1.23060,1.21220,1.22100,7593028.91
22153,2021-07-14 05:00:00,1.22080,1.22170,1.19800,1.20280,15978748.32
22154,2021-07-14 06:00:00,1.20280,1.22000,1.19980,1.20740,7789568.81
22155,2021-07-14 07:00:00,1.20750,1.22410,1.20660,1.22200,6168385.93


In [240]:
start = pd.to_datetime('2021-04-01')
end = pd.to_datetime('2021-04-30')


ada2021 = ada15.loc[(ada15['date']> start)& (ada15['date']< end)].copy()
ada2021['RSI'] = RSI(ada2021['close'])
ada2021['RSI4']=RSI(ada2021['close'],56)
ada2021['RSI8']=RSI(ada2021['close'],112)
ada2021['STRSI'] = STRSIA(ada2021['close'])[0]
ada2021['STRSIK'] = STRSIA(ada2021['close'])[1]
ada2021['STRSID'] = STRSIA(ada2021['close'])[2]

ada2021['STRSI4'] = STRSIA(ada2021['close'],56,12,12)[0]
ada2021['STRSIK4'] = STRSIA(ada2021['close'],56,12,12)[1]
ada2021['STRSID4'] = STRSIA(ada2021['close'],56,12,12)[2]

ada2021['STRSI8'] = STRSIA(ada2021['close'],112,24,24)[0]
ada2021['STRSIK8'] = STRSIA(ada2021['close'],112,24,24)[1]
ada2021['STRSID8'] = STRSIA(ada2021['close'],112,24,24)[2]



ada2021

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,STRSIK,STRSID,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8
19658,2021-04-01 01:00:00,1.19465,1.19635,1.18705,1.19015,8758248.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19659,2021-04-01 02:00:00,1.19011,1.19266,1.18870,1.19127,3718493.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19660,2021-04-01 03:00:00,1.19138,1.19149,1.18590,1.18779,4634667.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19661,2021-04-01 04:00:00,1.18780,1.19924,1.18637,1.19403,7909866.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19662,2021-04-01 05:00:00,1.19403,1.19869,1.18900,1.18923,6682481.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20343,2021-04-29 19:00:00,1.30650,1.30770,1.29550,1.29830,14712037.49,41.531731,51.425497,51.385816,0.049801,0.051758,0.060800,0.021001,0.475255,0.612051,0.614720,0.839729,0.837415
20344,2021-04-29 20:00:00,1.29860,1.30980,1.26610,1.29650,37268500.21,41.170632,51.310419,51.329661,0.037239,0.064171,0.050362,0.006367,0.403583,0.598283,0.608621,0.830225,0.837804
20345,2021-04-29 21:00:00,1.29650,1.30980,1.28380,1.30930,12872475.77,44.843175,52.086727,51.708319,0.190398,0.092479,0.069470,0.105089,0.345169,0.578794,0.649746,0.818652,0.837712
20346,2021-04-29 22:00:00,1.30920,1.31460,1.30010,1.31400,15373611.48,46.171957,52.370640,51.847118,0.247001,0.158213,0.104954,0.141194,0.284669,0.551502,0.664821,0.810932,0.837378


In [223]:
#maxlevels = [13,13,13,13,13,13,13]
#maxlevels = np.array(maxlevels)
#maxlevels

In [224]:
#minlevels = [11,11,11,11,11,11,11]
#minlevels = np.array(minlevels)
#minlevels

In [225]:
ada2021

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSIK,STRSIK4,STRSIK8,STRSID,STRSID4,STRSID8
19658,2021-04-01 01:00:00,1.19465,1.19635,1.18705,1.19015,8758248.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19659,2021-04-01 02:00:00,1.19011,1.19266,1.18870,1.19127,3718493.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19660,2021-04-01 03:00:00,1.19138,1.19149,1.18590,1.18779,4634667.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19661,2021-04-01 04:00:00,1.18780,1.19924,1.18637,1.19403,7909866.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19662,2021-04-01 05:00:00,1.19403,1.19869,1.18900,1.18923,6682481.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20343,2021-04-29 19:00:00,1.30650,1.30770,1.29550,1.29830,14712037.49,41.531731,51.425497,51.385816,0.051758,0.475255,0.839729,0.060800,0.612051,0.837415
20344,2021-04-29 20:00:00,1.29860,1.30980,1.26610,1.29650,37268500.21,41.170632,51.310419,51.329661,0.064171,0.403583,0.830225,0.050362,0.598283,0.837804
20345,2021-04-29 21:00:00,1.29650,1.30980,1.28380,1.30930,12872475.77,44.843175,52.086727,51.708319,0.092479,0.345169,0.818652,0.069470,0.578794,0.837712
20346,2021-04-29 22:00:00,1.30920,1.31460,1.30010,1.31400,15373611.48,46.171957,52.370640,51.847118,0.158213,0.284669,0.810932,0.104954,0.551502,0.837378


In [241]:
high = ada2021.high
low = ada2021.low
maximums5 = high.rolling(5,5)
max5 = maximums5.max()
minimums5 = low.rolling(5,5)
min5 = minimums5.min()
maximums10 = high.rolling(10,10)
max10 = maximums10.max()
minimums10 = low.rolling(10,10)
min10 = minimums10.min()


In [242]:
len(max5)

690

In [243]:

ada2021['max5'] = max5
ada2021['max10'] = max10
ada2021['min5'] = min5
ada2021['min10'] = min10

ada2021.tail()

,date,open,high,low,close,volume,RSI,RSI4,RSI8,STRSI,...,STRSI4,STRSIK4,STRSID4,STRSI8,STRSIK8,STRSID8,max5,max10,min5,min10
20343,2021-04-29 19:00:00,1.3065,1.3077,1.2955,1.2983,14712037.49,41.531731,51.425497,51.385816,0.049801,...,0.021001,0.475255,0.612051,0.614720,0.839729,0.837415,1.3688,1.4117,1.2800,1.2800
20344,2021-04-29 20:00:00,1.2986,1.3098,1.2661,1.2965,37268500.21,41.170632,51.310419,51.329661,0.037239,...,0.006367,0.403583,0.598283,0.608621,0.830225,0.837804,1.3598,1.3997,1.2661,1.2661
20345,2021-04-29 21:00:00,1.2965,1.3098,1.2838,1.3093,12872475.77,44.843175,52.086727,51.708319,0.190398,...,0.105089,0.345169,0.578794,0.649746,0.818652,0.837712,1.3232,1.3959,1.2661,1.2661
20346,2021-04-29 22:00:00,1.3092,1.3146,1.3001,1.3140,15373611.48,46.171957,52.370640,51.847118,0.247001,...,0.141194,0.284669,0.551502,0.664821,0.810932,0.837378,1.3146,1.3837,1.2661,1.2661
20347,2021-04-29 23:00:00,1.3140,1.3178,1.3055,1.3063,10226440.56,44.289468,51.858057,51.601947,0.170421,...,0.076009,0.221820,0.517519,0.638193,0.799544,0.836585,1.3178,1.3773,1.2661,1.2661


In [244]:
maxtest = np.ones(10)
maxtest = (maxtest * maxlevel)
mintest = mintest * 11
maxtest.shape
maxtest
mintest.astype(float)
maxtest.astype(float)
print(mintest.shape)
mintest.reshape(1,10)
print(mintest)
mintest.shape

NameError: name 'mintest' is not defined

In [245]:
#maxnew = np.ones_like(ratios)
#maxnew

In [246]:
#addmin = mintest.tolist()
#addmax = maxtest.tolist()
#data = pd.DataFrame({"max":maxtest,"min":mintest},dtype='float')
#data = data.append(addmin,ignore_index=True)
#data1 = pd.DataFrame(maxtest)
#data2 = pd.DataFrame(mintest)
#data = pd.DataFrame(data1,data2)

In [247]:
#datamul = data.mul([2,1],axis='columns')
#datamul = data['max'].mul(ratios[7])

In [248]:
#datamul

In [249]:
#df = ada2021.loc[(ada2021['max10']>0)& (ada2021['min10']>0)]
df = ada2021
#len(df['date'])
average = df['max5']-df['min5']
avg = average.mean()
avg
df
#point = df.loc[(df['max5'].diff()>0)]
#pointm = df.loc[(df['min5'].diff()>0)]

df['support5']= df.min5.rolling(5,5).min()
df['support10']= df.min10.rolling(10,10).min()
df['resist5']= df.max5.rolling(5,5).max()
df['resist10']= df.max10.rolling(10,10).max()

#df['RSI'] = RSI(df['close'])
#df['STRSI4']=STRSIA(df['close'],56,12,12)
#df['STRSI8']=STRSIA(df['close'],112,24,24)


point = df[(df['max5'] > df['max5'].shift(1)) &  (df['max5'].shift(1) > df['max5'].shift(2))]
pointm = df[(df['min5'] < df['min5'].shift(1)) &  (df['min5'].shift(1) > df['min5'].shift(2))]
point
#ada2021
pointm

for row in range(0,len(pointm)):
    print(pointm['support5'])

19674    1.18150
19681    1.17171
19693    1.19000
19720    1.20432
19736    1.15200
19797    1.19500
19809    1.22250
19819    1.15880
19834    1.17539
19845    1.18790
19852    1.20000
19863    1.20100
19872    1.19134
19883    1.19157
19891    1.20223
19902    1.19900
19937    1.26350
19949    1.29450
19967    1.36933
19978    1.38000
19995    1.37500
20004    1.42190
20011    1.42809
20018    1.44441
20039    1.35000
20050    1.41226
20065    1.35000
20097    1.25394
20112    1.17122
20126    1.12685
20136    1.19834
20145    1.21764
20155    1.20572
20167    1.17851
20191    0.92000
20207    1.08456
20217    1.11301
20230    1.09270
20246    1.10300
20305    1.27550
20337    1.29650
Name: support5, dtype: float64
19674    1.18150
19681    1.17171
19693    1.19000
19720    1.20432
19736    1.15200
19797    1.19500
19809    1.22250
19819    1.15880
19834    1.17539
19845    1.18790
19852    1.20000
19863    1.20100
19872    1.19134
19883    1.19157
19891    1.20223
19902    1.19900


In [255]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots


#Declare figure


fig = go.Figure()

fig = make_subplots(rows=8, cols=1,subplot_titles=("Price", "RSI 1 4 8", "Stoch 1", "Stoch 4","Stoch 8"))
    

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI'],
        name="RSI",
        text="RSI",
        textposition="top center"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI4'],name="RSI4"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['RSI8'],name="RSI8"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI'],name="STRSI"),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK'],name="STRSIK"),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID'],name="STRSID"),
    row=3, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI4'],name="STRSI4"),
    row=4, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK4'],name="STRSIK4"),
    row=4, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID4'],name="STRSID4"),
    row=4, col=1
)
    
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSI8'],name="STRSI8"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSIK8'],name="STRSIK8"),
    row=5, col=1
)
fig.add_trace(
    go.Scatter(x=df.date, y=df['STRSID8'],name="STRSIK8"),
    row=5, col=1
)
#add Traces

fig.add_trace(go.Candlestick(x=df.date,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'], name = 'market data'),row=1,col=1)

#fig.add_trace(go.Scatter(x=df['date'], y=df['max10']))

#fig.add_trace(go.Scatter(x=df['date'], y=df['min10']))

fig.add_trace(go.Scatter(x=point['date'], y=point['resist5']))

fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support5']))


#fig.add_shape(type="line",
#    x0=pointm['date'], y0=pointm['support5'], x1=max(pointm['date']), y1=pointm['support5'],
#    line=dict(
#        color="LightSeaGreen",
#        width=4,
#        dash="dashdot",
#    ))
fig.update_shapes(dict(xref='x', yref='y'))

#for index in range(0,len(pointm)):
    #fig.add_hline(y=pointm['support5'])

#fig.add_trace(go.Scatter(x=point['date'], y=point['resist10']))

#fig.add_trace(go.Scatter(x=pointm['date'], y=pointm['support10']))
#for i in range(len(df['date'])):
    #fig.add_hline(y=df[i]['min10'], line_dash="dot",
                  #annotation_text="support", 
                  #annotation_position="bottom right")

#fig.add_hline(y=df['max10'], line_dash="dot",
#              annotation_text="resitance", 
#              annotation_position="bottom right")



fig.update_layout(height=3000, width=800, title_text="top sub plots")
# X and Y Axis Update
#fig.update_xaxes(
#    title = 'Date',rangeslider_visible=True
#)

#fig.update_yaxes(
#    title = 'Share Price (USD)'
#)

#Show
fig.show()